# Hugging Face Transformers 微调训练入门

## Homework: 使用完整的 YelpReviewFull 数据集训练，看 Acc 最高能到多少
完整训练集训练3个epoch, save_strategy="steps", save_steps="500", save_total_limit="10", logging_steps="500"

### 下载/加载数据集

In [1]:
# 下载/加载数据集
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

### 预处理数据

In [2]:
# 预处理数据
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

/root/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### 设置训练集和测试集

In [3]:
train_dataset = tokenized_datasets['train']
test_dataset = tokenized_datasets['test']
print(f"train: {len(train_dataset)}; test: {len(test_dataset)}")

train: 650000; test: 50000


### 设置评估指标

In [4]:
# 设置评估指标
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

### 加载Bert模型

In [5]:
# 加载Bert模型
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

/root/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 设置训练超参数

In [6]:
# 训练超参数
from transformers import TrainingArguments, Trainer

model_dir = "models/bert-base-cased-finetune-yelp-2"
training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  num_train_epochs=3,
                                  logging_steps=500,
                                  save_total_limit=10
                                )

### 实例化训练器

In [7]:
# 实例化训练器
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

### 开始训练

In [8]:
# 开始训练
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.774200,0.756893,0.671340
2,0.697000,0.730391,0.684100
3,0.614300,0.729855,0.691780


TrainOutput(global_step=121875, training_loss=0.7127398907001202, metrics={'train_runtime': 171225.0172, 'train_samples_per_second': 11.389, 'train_steps_per_second': 0.712, 'total_flos': 5.130803778048e+17, 'train_loss': 0.7127398907001202, 'epoch': 3.0})

### 保存模型和训练状态

In [9]:
# 保存模型和训练状态
trainer.save_model(model_dir)
trainer.save_state()

In [10]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.7298545241355896,
 'eval_accuracy': 0.69178,
 'eval_runtime': 1508.2696,
 'eval_samples_per_second': 33.151,
 'eval_steps_per_second': 4.144,
 'epoch': 3.0}